In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import copy
import numpy as np
import os


import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
from glob import glob

from PIL import Image

use_cuda = torch.cuda.is_available()

In [2]:
def load_image(path, imsize=224, volatile=True, use_cuda=False):
    im = Image.open(path)
    im = RGBA2RGB(im)

    # crop to sketch only (eliminate white space)
    arr = np.asarray(im)
    w,h,d = np.where(arr<255) # where the image is not white
    if len(h)==0:
        print(path)            
    xlb = min(h)
    xub = max(h)
    ylb = min(w)
    yub = max(w)
    lb = min([xlb,ylb])
    ub = max([xub,yub])            
    im = im.crop((lb, lb, ub, ub))            

    loader = transforms.Compose([
        transforms.Pad(padding),                
        transforms.Scale(imsize),
        transforms.ToTensor()])

    im = Variable(loader(im), volatile=volatile)
    im = im.unsqueeze(0)
    return im 

        
def RGBA2RGB(image, color=(255, 255, 255)):
    image.load()  # needed for split()
    background = Image.new('RGB', image.size, color)
    background.paste(image, mask=image.split()[3])  # 3 is the alpha channel
    return background



In [3]:
def generator(paths, imsize):
    for path in paths:
        image = load_image(path)
        label, age, session = get_metadata_from_path(path)
        yield (image, label, age, session)       

def list_files(path, ext='png'):
        result = [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
        return result

def RGBA2RGB(image, color=(255, 255, 255)):
    """Alpha composite an RGBA Image with a specified color.
    Simpler, faster version than the solutions above.
    Source: http://stackoverflow.com/a/9459208/284318
    Keyword Arguments:
    image -- PIL RGBA Image object
    color -- Tuple r, g, b (default 255, 255, 255)
    """
    image.load()  # needed for split()
    background = Image.new('RGB', image.size, color)
    background.paste(image, mask=image.split()[3])  # 3 is the alpha channel
    return background

  

In [4]:
### scratch pad to outline pixel extraction
data_path = '/data2/jefan/quickDraw/png_micro'
all_pngs = list_files(data_path)
imSize=224
numPixels=imSize*imSize


In [5]:
def get_metadata_from_path(path,cohort):
    label = path.split('/')[-2]            
    if cohort == 'kid':
        age = path.split('/')[-1].split('_')[2]
        session = path.split('/')[-1].split('.')[0].split('_')[-2] + '_' + path.split('/')[-1].split('.')[0].split('_')[-1]
    elif cohort == 'adult':
        age = 'adult'
        session = 'unknown'
    else:
        print('Need to specify a cohort: "kid" or "adult"!')
        age = 'unknown'
        session = 'unknown'
    return label, age, session


In [6]:
## Adult features
Labels = []
Ages = []
Sessions = []
Features = np.zeros((len(all_pngs),numPixels))

for vi, v in enumerate(np.asarray(all_pngs)):
    im = Image.open(v)
    im = RGBA2RGB(im)
    im2 = im.resize((224,224), Image.ANTIALIAS)
    arr = np.array(im2)
    oneChannel = arr[:,:,1];
    pixels = np.ravel(oneChannel)
    Features[vi,:] = pixels
    label, age, session = get_metadata_from_path(v,'adult')
    Labels.append(label)
    Ages.append(age)
    Sessions.append(session)
    


In [7]:
## Save adult pixels
save_path = '/home/bria/kiddraw/analysis/museumdraw/python/features/FEATURES_pixel_adult'
np.save(save_path, Features)

import pandas as pd
df = pd.DataFrame({
    'Labels':  Labels,
    'Ages': Ages,
    'Sessions': Sessions
})

save_path_csv = '/home/bria/kiddraw/analysis/museumdraw/python/features/METADATA_ADULT_Pixels'
df.to_csv(save_path_csv) 

In [8]:
## Now for kid features
data_path_kids = '/home/jefan/kiddraw/analysis/museumdraw/sketches'
all_pngs_kids = list_files(data_path_kids)

all_pngs_kids[0]

'/home/jefan/kiddraw/analysis/museumdraw/sketches/banana/banana_sketch_8_E1c_1511300355318.png'

In [9]:
## Kid features
Labels = []
Ages = []
Sessions = []
Features = np.zeros((len(all_pngs_kids),numPixels))

for vi, v in enumerate(np.asarray(all_pngs_kids)):
    im = Image.open(v)
    im = RGBA2RGB(im)
    im2 = im.resize((224,224), Image.ANTIALIAS)
    arr = np.array(im2)
    oneChannel = arr[:,:,1];
    pixels = np.ravel(oneChannel)
    Features[vi,:] = pixels
    label, age, session = get_metadata_from_path(v,'kid')
    Labels.append(label)
    Ages.append(age)
    Sessions.append(session)
    

In [10]:
save_path_kid = '/home/bria/kiddraw/analysis/museumdraw/python/features/FEATURES_pixel_kid'
np.save(save_path_kid, Features)

import pandas as pd
df = pd.DataFrame({
    'Labels':  Labels,
    'Ages': Ages,
    'Sessions': Sessions
})

save_path_csv_kid = '/home/bria/kiddraw/analysis/museumdraw/python/features/METADATA_KID_Pixels'
df.to_csv(save_path_csv_kid) 